In [1]:
%matplotlib qt
import src
from src.sweep0d import Sweep0D
from src.sweep1d import Sweep1D
from src.sweep2d import Sweep2D
from src.sweep_queue import SweepQueue, DatabaseEntry
from src.daq_driver import Daq
from src.util import init_database
from src.tracking import *
import nidaqmx
import time, os
import numpy as np
import qcodes as qc
from qcodes import Station
from qcodes.dataset.measurements import Measurement
from qcodes import initialise_or_create_database_at
from qcodes.dataset.data_set import load_by_run_spec
from qcodes.dataset.data_export import get_data_by_id
from qcodes.instrument_drivers.stanford_research.SR830 import SR830
from qcodes.instrument_drivers.stanford_research.SR860 import SR860

In [2]:
station = Station(config_file='cfg/dacen_21_1_20.station.yaml')

In [3]:
station.snapshot()

{'instruments': {},
 'parameters': {},
 'components': {},
 'config': {'instruments': ordereddict([('Daq', ordereddict([('address', 'Dev1'), ('enable_forced_reconnect', True), ('type', 'src.daq_driver.Daq')])), ('keithley_bg', ordereddict([('address', 'USB0::0x05E6::0x2450::04428921::INSTR'), ('enable_forced_reconnect', True), ('type', 'qcodes.instrument_drivers.tektronix.Keithley_2450.Keithley2450')])), ('keithley_sg', ordereddict([('address', 'USB0::0x05E6::0x2450::04453530::INSTR'), ('enable_forced_reconnect', True), ('type', 'qcodes.instrument_drivers.tektronix.Keithley_2450.Keithley2450')])), ('keithley_tg', ordereddict([('address', 'USB0::0x05E6::0x2450::04428406::INSTR'), ('enable_forced_reconnect', True), ('type', 'qcodes.instrument_drivers.tektronix.Keithley_2450.Keithley2450')])), ('ls340', ordereddict([('address', 'GPIB0::12::INSTR'), ('enable_forced_reconnect', True), ('type', 'src.Model_340.Model_340')])), ('sr830', ordereddict([('address', 'GPIB0::8::INSTR'), ('enable_forc

In [4]:
station.snapshot()['config']['instruments'].items()

In [5]:
instruments = {}
for (name, other) in station.snapshot()['config']['instruments'].items():
    instruments[name] = station.load_instrument(name)

[Daq_ao0(DaqAOChannel)] Snapshot: Could not update parameter: gain_factor
[Daq_ao0(DaqAOChannel)] Snapshot: Could not update parameter: impedance
[Daq_ao0(DaqAOChannel)] Snapshot: Could not update parameter: value
[Daq_ao1(DaqAOChannel)] Snapshot: Could not update parameter: gain_factor
[Daq_ao1(DaqAOChannel)] Snapshot: Could not update parameter: impedance
[Daq_ao1(DaqAOChannel)] Snapshot: Could not update parameter: value
[Daq_ao2(DaqAOChannel)] Snapshot: Could not update parameter: gain_factor
[Daq_ao2(DaqAOChannel)] Snapshot: Could not update parameter: impedance
[Daq_ao2(DaqAOChannel)] Snapshot: Could not update parameter: value
[Daq_ao3(DaqAOChannel)] Snapshot: Could not update parameter: gain_factor
[Daq_ao3(DaqAOChannel)] Snapshot: Could not update parameter: impedance
[Daq_ao3(DaqAOChannel)] Snapshot: Could not update parameter: value
[Daq_ai0(DaqAIChannel)] Snapshot: Could not update parameter: gain_factor
[Daq_ai0(DaqAIChannel)] Snapshot: Could not update parameter: impedanc

Connected to: NI DAQ USB-6259 (BNC) (Dev1) in 0.15s


[Daq_ai4(DaqAIChannel)] Snapshot: Could not update parameter: gain_factor
[Daq_ai4(DaqAIChannel)] Snapshot: Could not update parameter: impedance
[Daq_ai4(DaqAIChannel)] Snapshot: Could not update parameter: value
[Daq_ai5(DaqAIChannel)] Snapshot: Could not update parameter: gain_factor
[Daq_ai5(DaqAIChannel)] Snapshot: Could not update parameter: impedance
[Daq_ai5(DaqAIChannel)] Snapshot: Could not update parameter: value
[Daq_ai6(DaqAIChannel)] Snapshot: Could not update parameter: gain_factor
[Daq_ai6(DaqAIChannel)] Snapshot: Could not update parameter: impedance
[Daq_ai6(DaqAIChannel)] Snapshot: Could not update parameter: value
[Daq_ai7(DaqAIChannel)] Snapshot: Could not update parameter: gain_factor
[Daq_ai7(DaqAIChannel)] Snapshot: Could not update parameter: impedance
[Daq_ai7(DaqAIChannel)] Snapshot: Could not update parameter: value
[Daq_ai16(DaqAIChannel)] Snapshot: Could not update parameter: gain_factor
[Daq_ai16(DaqAIChannel)] Snapshot: Could not update parameter: impeda

Connected to: KEITHLEY INSTRUMENTS 2450 (serial:04428921, firmware:1.6.7c) in 0.04s


[keithley_bg_source(Source2450)] Snapshot: Could not update parameter: user_delay
[keithley_bg_source(Source2450)] Snapshot: Could not update parameter: sweep_axis
[keithley_bg_source(Source2450)] Snapshot: Could not update parameter: user_delay
[keithley_bg_sense(Sense2450)] Snapshot: Could not update parameter: user_delay
[keithley_bg_sense(Sense2450)] Snapshot: Could not update parameter: user_delay
[keithley_bg_sense(Sense2450)] Snapshot: Could not update parameter: user_delay
[keithley_sg_source(Source2450)] Snapshot: Could not update parameter: sweep_axis


...

Connected to: KEITHLEY INSTRUMENTS 2450 (serial:04428406, firmware:1.6.7c) in 0.02s


[keithley_tg_source(Source2450)] Snapshot: Could not update parameter: user_delay
[keithley_tg_source(Source2450)] Snapshot: Could not update parameter: sweep_axis
[keithley_tg_source(Source2450)] Snapshot: Could not update parameter: user_delay
[keithley_tg_sense(Sense2450)] Snapshot: Could not update parameter: user_delay
[keithley_tg_sense(Sense2450)] Snapshot: Could not update parameter: user_delay
[keithley_tg_sense(Sense2450)] Snapshot: Could not update parameter: user_delay
[ls340_A(Model_340_Channel)] Snapshot: Could not update parameter: sensor_type
[ls340_A(Model_340_Channel)] Snapshot: Could not update parameter: compensation_enabled


Connected to: LSCI 340 (serial:000000, firmware:021901) in 0.07s


[ls340_B(Model_340_Channel)] Snapshot: Could not update parameter: sensor_type
[ls340_B(Model_340_Channel)] Snapshot: Could not update parameter: compensation_enabled
[ls340_output_1(Output_340)] Snapshot: Could not update parameter: mode
[ls340_output_1(Output_340)] Snapshot: Could not update parameter: input_channel
[ls340_output_1(Output_340)] Snapshot: Could not update parameter: units
[ls340_output_1(Output_340)] Snapshot: Could not update parameter: current_or_power
[ls340_output_1(Output_340)] Snapshot: Could not update parameter: powerup_enable
[ls340_output_2(Output_340)] Snapshot: Could not update parameter: mode
[ls340_output_2(Output_340)] Snapshot: Could not update parameter: input_channel
[ls340_output_2(Output_340)] Snapshot: Could not update parameter: units
[ls340_output_2(Output_340)] Snapshot: Could not update parameter: current_or_power
[ls340_output_2(Output_340)] Snapshot: Could not update parameter: powerup_enable


Connected to: Stanford_Research_Systems SR830 (serial:s/n31208, firmware:ver1.07) in 0.04s
Connected to: Stanford_Research_Systems SR860 (serial:003594, firmware:V1.51) in 0.04s


In [6]:
print(instruments)
daq = instruments['Daq']
keithley_bg = instruments['keithley_bg']
keithley_tg = instruments['keithley_tg']
keithley_sg = instruments['keithley_sg']
ls340 = instruments['ls340']
sr830 = instruments['sr830']
sr860 = instruments['sr860']

{'Daq': <Daq: Daq>, 'keithley_bg': <Keithley2450: keithley_bg>, 'keithley_sg': <Keithley2450: keithley_sg>, 'keithley_tg': <Keithley2450: keithley_tg>, 'ls340': <Model_340: ls340>, 'sr830': <SR830: sr830>, 'sr860': <SR860: sr860>}


In [7]:
daq.ai0.voltage.label = 'Rxx_T5-T6'
daq.ai1.voltage.label = 'Rxx_p_T5-T6'
daq.ai2.voltage.label = 'Rxx_T2-T3'
daq.ai3.voltage.label = 'Rxx_p_T2-T3'
daq.ai4.voltage.label = 'Rxx_B1-B2'
daq.ai5.voltage.label = 'Rxx_p_B1-B2'
keithley_bg.sense.current.label = 'I_bg'
keithley_tg.sense.current.label = 'I_tg'
sr830.X.label = 'Rxy_T2-T5'
sr830.P.label = 'Rxy_p_T2-T5'
sr860.X.label = 'I_s'
sr860.P.label = 'I_s_p'

In [8]:
ramp_bg = Sweep1D(keithley_bg.source.voltage, 0, -13, -.1, inter_delay=0.25, save_data=False, x_axis_time = False)
ramp_bg.follow_param(daq.ai0.voltage, daq.ai1.voltage, daq.ai2.voltage, daq.ai3.voltage, daq.ai4.voltage, daq.ai5.voltage, sr830.X, sr830.P, sr860.X, sr860.P)
ramp_tg = Sweep1D(keithley_tg.source.voltage, 0, -8, -.1, inter_delay=0.25, save_data=False, x_axis_time = False)
ramp_tg.follow_param(daq.ai0.voltage, daq.ai1.voltage, daq.ai2.voltage, daq.ai3.voltage, daq.ai4.voltage, daq.ai5.voltage, sr830.X, sr830.P, sr860.X, sr860.P)
ramp_sg = Sweep1D(keithley_sg.source.voltage, 0, 10, -.1, inter_delay=0.25, save_data=False, x_axis_time = False)
ramp_sg.follow_param(daq.ai0.voltage, daq.ai1.voltage, daq.ai2.voltage, daq.ai3.voltage, daq.ai4.voltage, daq.ai5.voltage, sr830.X, sr830.P, sr860.X, sr860.P)

In [9]:
db = os.environ['MeasureItHome']+'\\Databases\\dw_14-DtBLG.db'

In [10]:
dV = np.diff(np.linspace(-13, 13, 300, endpoint=True))[0]
print(dV)
#print(np.linspace(-13, 13, 400, endpoint=True))

0.08695652173913082


In [11]:
sweep_queue = SweepQueue(inter_delay = 1)
exp_name = '210127-2d-gate-sweep_B=12T_T=3.6K_V_sg=10V'

for V_bg in np.linspace(-13, 13, 300, endpoint=False):
    
    # Add new gate sweeps at that voltage
    forward_gate = Sweep1D(keithley_tg.source.voltage, -8, 8, .05, inter_delay=.5, x_axis_time=0)
    forward_gate.follow_param(daq.ai0.voltage, daq.ai1.voltage, daq.ai2.voltage, daq.ai3.voltage, daq.ai4.voltage, daq.ai5.voltage, sr830.X, sr830.P, sr860.X, sr860.P)
    backward_gate = Sweep1D(keithley_tg.source.voltage, 8, -8, -.2, inter_delay=.25, save_data=False, x_axis_time=0)
    backward_gate.follow_param(daq.ai0.voltage, daq.ai1.voltage, daq.ai2.voltage, daq.ai3.voltage, daq.ai4.voltage, daq.ai5.voltage, sr830.X, sr830.P, sr860.X, sr860.P)
    
    # Create the database entry
    sample_name = f'V_bg={V_bg:.2f}V_sweep-Vtg'
    db_entry = DatabaseEntry(db, exp_name, sample_name)
    sweep_queue.append(db_entry)
    sweep_queue.append(forward_gate, backward_gate)

    # Step the back gate field
    
    #vbg_step = Sweep1D(keith_tg.source.voltage, V_tg, V_tg+.08, .04, inter_delay=.1, x_axis_time=0, save_data=False)
    vbg_step = Sweep1D(keithley_bg.source.voltage, V_bg, V_bg+dV, 0.04, inter_delay=0.1, x_axis_time=0, save_data=False)
    vbg_step.follow_param(daq.ai0.voltage, daq.ai1.voltage, daq.ai2.voltage, daq.ai3.voltage, daq.ai4.voltage, daq.ai5.voltage, sr830.X, sr830.P, sr860.X, sr860.P)
    
    # Add it to our queue
    sweep_queue.append(vbg_step)
    
# Add last gate sweep
forward_gate = Sweep1D(keithley_tg.source.voltage, -8, 8, .05, inter_delay=.5, x_axis_time=0)
backward_gate = Sweep1D(keithley_tg.source.voltage, 8, 0, -.2, inter_delay=.25, save_data=False, x_axis_time=0)
forward_gate.follow_param(daq.ai0.voltage, daq.ai1.voltage, daq.ai2.voltage, daq.ai3.voltage, daq.ai4.voltage, daq.ai5.voltage, sr830.X, sr830.P, sr860.X, sr860.P)
backward_gate.follow_param(daq.ai0.voltage, daq.ai1.voltage, daq.ai2.voltage, daq.ai3.voltage, daq.ai4.voltage, daq.ai5.voltage, sr830.X, sr830.P, sr860.X, sr860.P)
    
# Create the database entry
sample_name = 'V_bg=13.00V_sweep-Vtg'
db_entry = DatabaseEntry(db, exp_name, sample_name)
sweep_queue.append(db_entry)
sweep_queue.append(forward_gate, backward_gate)

# Bring the back gate back to 0
vbg_step = Sweep1D(keithley_bg.source.voltage, 13, 0, .04, inter_delay=.1, x_axis_time=0, save_data=False)
vbg_step.follow_param(daq.ai0.voltage, daq.ai1.voltage, daq.ai2.voltage, daq.ai3.voltage, daq.ai4.voltage, daq.ai5.voltage, sr830.X, sr830.P, sr860.X, sr860.P)
sweep_queue.append(vbg_step)

# Bring the silicon gate back to 0
vsg_step = Sweep1D(keithley_sg.source.voltage, 10, 0, .04, inter_delay=.1, x_axis_time=0, save_data=False)
vsg_step.follow_param(daq.ai0.voltage, daq.ai1.voltage, daq.ai2.voltage, daq.ai3.voltage, daq.ai4.voltage, daq.ai5.voltage, sr830.X, sr830.P, sr860.X, sr860.P)
sweep_queue.append(vsg_step)

In [12]:
for item in sweep_queue.queue:
    print(item)

Database entry saving to C:\Users\nanouser\Documents\MeasureIt\Databases\dw_14-DtBLG.db with experiment name 210127-2d-gate-sweep_B=12T_T=3.6K_V_sg=10V and sample name V_bg=-13.00V_sweep-Vtg.
1D Sweep of keithley_tg_source_voltage from -8 to 8, with step size 0.05.
1D Sweep of keithley_tg_source_voltage from 8 to -8, with step size -0.2.
1D Sweep of keithley_bg_source_voltage from -13.0 to -12.91304347826087, with step size 0.04.
Database entry saving to C:\Users\nanouser\Documents\MeasureIt\Databases\dw_14-DtBLG.db with experiment name 210127-2d-gate-sweep_B=12T_T=3.6K_V_sg=10V and sample name V_bg=-12.91V_sweep-Vtg.
1D Sweep of keithley_tg_source_voltage from -8 to 8, with step size 0.05.
1D Sweep of keithley_tg_source_voltage from 8 to -8, with step size -0.2.
1D Sweep of keithley_bg_source_voltage from -12.913333333333334 to -12.826376811594203, with step size 0.04.
Database entry saving to C:\Users\nanouser\Documents\MeasureIt\Databases\dw_14-DtBLG.db with experiment name 210127-2

In [13]:
# ramp to start point
ramp_bg.start()

Ramping to our starting setpoint value of 0 V
Sweeping voltage to 0 V
Ramping voltage to 0 . . . 
Done ramping voltage to 0
Sweeping voltage to -13 V


In [14]:
ramp_tg.start()

Ramping to our starting setpoint value of 0 V
Sweeping voltage to 0 V
Ramping voltage to 0 . . . 
Done ramping voltage to 0
Sweeping voltage to -8 V


In [15]:
ramp_sg.start()

Ramping to our starting setpoint value of 0 V
Done ramping voltage to 0
Sweeping voltage to 10 V


In [16]:
sweep_queue.start()

Starting sweeps
Starting sweep of voltage from -8                           V to 8 V
Ramping to our starting setpoint value of -8 V
Done ramping voltage to -8
Sweeping voltage to 8 V
Starting experimental run with id: 583. 
Finished sweep of voltage from 8                   V to -8 V
Starting sweep of voltage from 8                           V to -8 V
Ramping to our starting setpoint value of 8 V
Sweeping voltage to 8 V
Ramping voltage to 8 . . . 
Done ramping voltage to 8
Sweeping voltage to -8 V
Finished sweep of voltage from -8                   V to 8 V
Starting sweep of voltage from -13.0                           V to -12.91304347826087 V
Ramping to our starting setpoint value of -13.0 V
Done ramping voltage to -13.0
Sweeping voltage to -12.91304347826087 V
Finished sweep of voltage from -12.91304347826087                   V to -13.0 V
Starting sweep of voltage from -8                           V to 8 V
Ramping to our starting setpoint value of -8 V
Done ramping voltage to -8
Sw

In [52]:
print(sweep_queue.current_sweep)

1D Sweep of Daq_ao1_voltage from 9.733333333333334 to 10.000000000000002, with step size 0.04.


In [62]:
ls340.A.temperature()

3.1741

In [ ]:
ramp_tg.kill()

In [ ]:
sr830.X()

In [89]:
sweep_queue.current_sweep.kill()

In [ ]:
print('test')

In [29]:
daq.ao1.voltage()

0.0001750530144525011

In [84]:
daq.ao0.voltage()

-0.0003854927010529517

In [85]:
daq.ai6.voltage()

-7.999318267642534

In [86]:
daq.ai7.voltage()

-10.556968012612895

In [8]:
s=Sweep1D(daq.ao1.voltage, 0, 10, 0.1, inter_delay=.1, save_data=False)
s.follow_param(daq.ai7.voltage)

In [9]:
s.start(ramp_to_start=False)

Sweeping ao1 Voltage to 10 V
Sweeping ao1 Voltage to 0 V


In [41]:
s.kill()

In [69]:
daq.ao1.voltage()

-10.000262914981269

In [45]:
ramp_tg.kill()

In [70]:
vtg_step = Sweep1D(daq.ao1.voltage, -10, 0, 0.04, inter_delay=.05, x_axis_time=0, save_data=False)
vtg_step.follow_param(daq.ai7.voltage)

In [71]:
vtg_step.start()

Ramping to our starting setpoint value of -10 V
Done ramping ao1 Voltage to -10
Sweeping ao1 Voltage to 0 V


In [105]:
ls340.A.temperature()

3.5612

Finished sweep of voltage from 8                   V to -8 V
Starting sweep of voltage from 8                           V to -8 V
Ramping to our starting setpoint value of 8 V
Sweeping voltage to 8 V
Ramping voltage to 8 . . . 
Done ramping voltage to 8
Sweeping voltage to -8 V
Finished sweep of voltage from -8                   V to 8 V
Starting sweep of voltage from -1.7333333333333325                           V to -1.6463768115942017 V
Ramping to our starting setpoint value of -1.7333333333333325 V
Done ramping voltage to -1.7333333333333325
Sweeping voltage to -1.6463768115942017 V
Finished sweep of voltage from -1.6463768115942017                   V to -1.7333333333333325 V
Starting sweep of voltage from -8                           V to 8 V
Ramping to our starting setpoint value of -8 V
Done ramping voltage to -8
Sweeping voltage to 8 V
Starting experimental run with id: 270. 
Finished sweep of voltage from 8                   V to -8 V
Starting sweep of voltage from 8         

In [20]:
ls340.A.temperature()

3.409

Finished sweep of voltage from 8                   V to -8 V
Starting sweep of voltage from 8                           V to -8 V
Ramping to our starting setpoint value of 8 V
Sweeping voltage to 8 V
Ramping voltage to 8 . . . 
Done ramping voltage to 8
Sweeping voltage to -8 V
Finished sweep of voltage from -8                   V to 8 V
Starting sweep of voltage from 7.886666666666667                           V to 7.9736231884057975 V
Ramping to our starting setpoint value of 7.886666666666667 V
Done ramping voltage to 7.886666666666667
Sweeping voltage to 7.9736231884057975 V
Finished sweep of voltage from 7.9736231884057975                   V to 7.886666666666667 V
Starting sweep of voltage from -8                           V to 8 V
Ramping to our starting setpoint value of -8 V
Done ramping voltage to -8
Sweeping voltage to 8 V
Starting experimental run with id: 524. 
Finished sweep of voltage from 8                   V to -8 V
Starting sweep of voltage from 8                    